In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#!pip install keras
#!pip install --upgrade tensorflow
#!pip install --upgrade tensorflow-gpu
!pip install xlrd==1.2.0
!pip install openpyxl
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import os
from time import time
from collections import Counter

from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling1D, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Reshape
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score


from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [ ]:
data = pd.read_csv('SBTC - Sheet1.csv')

In [ ]:
print("Categories:")
print(data.final_label.unique())
category_list=data.final_label.unique()

Categories:
['Physics' 'Chemistry' 'Biology' 'ICT' 'Mathematics' 'Others']


In [ ]:
print("First rows of dataset")
data.head()

First rows of dataset


,ID,text,source,cleantext,label1,label2,label3,final_label
0,1,ভৌত জগৎ মূলত চারটি উপাদানের সমন্বয়ে তৈরি- (১)...,HSC Physics,ভৌত জগৎ মূলত চারটি উপাদানের সমন্বয়ে তৈরি স্থা...,Physics,Physics,Physics,Physics
1,2,বলবিদ্যা বস্তুর উপর বলের ক্রিয়া সংক্রান্ত বিভ...,HSC Physics,বলবিদ্যা বস্তুর উপর বলের ক্রিয়া সংক্রান্ত বিভ...,Physics,Physics,Physics,Physics
2,3,পদার্থের ধর্ম বস্তুর বিভিন্ন গুণ আলোচনা করে।,HSC Physics,পদার্থের ধর্ম বস্তুর বিভিন্ন গুণ আলোচনা করে,Physics,Physics,Physics,Physics
3,4,স্থিতিবিদ্যা স্থিতিশীল বস্তুর উপর বলের ক্রিয়া...,HSC Physics,স্থিতিবিদ্যা স্থিতিশীল বস্তুর উপর বলের ক্রিয়া...,Physics,Physics,Physics,Physics
4,5,গতিবিদ্যা গতিশীল বস্তুর উপর বলের ক্রিয়া আলোচন...,HSC Physics,গতিবিদ্যা গতিশীল বস্তুর উপর বলের ক্রিয়া আলোচন...,Physics,Physics,Physics,Physics


In [ ]:
print("Description of dataset")
data.describe()

Description of dataset


,ID
count,6000.000000
mean,3000.500000
std,1732.195139
min,1.000000
25%,1500.750000
50%,3000.500000
75%,4500.250000
max,6000.000000


In [ ]:
print("Dataframe Information on dataset")
data.info()

Dataframe Information on dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           6000 non-null   int64 
 1   text         6000 non-null   object
 2   source       6000 non-null   object
 3   cleantext    6000 non-null   object
 4   label1       6000 non-null   object
 5   label2       6000 non-null   object
 6   label3       6000 non-null   object
 7   final_label  6000 non-null   object
dtypes: int64(1), object(7)
memory usage: 375.1+ KB


In [ ]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)
    text = re.sub(r'₀|₁|₂|₃|₄|₅|₆|₇|₈|₉', '', text)
    text = re.sub(r'⁰|¹|²|³|⁴|⁵|⁶|⁷|⁸|⁹|⁻|⁺|ⁿ|ᵗ', '', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-−[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰θ→°µσπ∈∝α≤≥Δβε≠�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [ ]:
data['cleantext'] = data.text.apply(lambda x: preprocessing(str(x)))
# print some cleaned reviews from the dataset
sample_data = range(5000,6000)
for i in sample_data:
  print(data.cleantext[i])

সনাতন বা আদিম সমাজে মানুষের জীবনযাপন ছিল অত্যন্ত সহজ সরল 
খাবার দাবার কাপড় চোপড় এবং বাড়িঘর এসবই ছিল মানুষের মৌলিক চাহিদা 
মূলত মানুষের কায়িক পরিশ্রম ছিল উৎপাদনের একমাত্র উপকরণ 
দশে মিলে করি কাজ হারিজিতি নাহি লাজ এই ছিল আদিম সমাজের মূলমন্ত্র
উৎপাদন ভোগ ও দৈনন্দিন সংসার পরিচালনার বিদ্যাকেই তখন অর্থনীতি বলা হতো
প্রাচীন ভারতে চতুর্থ খ্রিষ্টপূর্বে কৌটিল্যের অর্থশাস্ত্রে বৃহত্তর পরিসরে সারা দেশের রাজনীতি সমাজ অর্থনীতি ও সামরিক বিষয়ের উপর আলোকপাত করা হয়
দেশের ধনসম্পদ বৃদ্ধি রাষ্ট্রের ক্ষমতা বৃদ্ধি ও বাণিজ্য উদ্বৃত্তকরণের লক্ষ্যে ইংল্যান্ডের ব্যবসায়ীরা বেশি রপ্তানি করত এবং খুব সামান্যই আমদানি করত
ইংল্যান্ডের উৎপাদিত পণ্য পৃথিবীর বিভিন্ন দেশে প্রচুর পরিমাণে রপ্তানি করে মূল্যবান ধাতু সোনা রূপা হীরা ইত্যাদি আমদানি করা হতো 
অষ্টাদশ শতাব্দীর মধ্যভাগে ফরাসিরা সে দেশের ধনী মানুষের বিলাসী জীবনযাপন অতিরিক্ত করারোপ এবং ইংল্যান্ডের বাণিজ্যবাদের বিপক্ষে অবস্থান নিয়ে ভূমিবাদ মতবাদ প্রচার করেন 
ভূমিবাদীদের মতে কৃষিই খনি ও মৎসক্ষেত্রসহ হলো উৎপাদনশীল খাত
অন্যদিকে শিল্প ও বাণিজ্য উভয়ই অনুৎপাদনশীল খাত 

In [ ]:
stop =["অতএব","অথচ","অথবা","অনুযায়ী","অনেক","অনেকে","অনেকেই","অন্তত","অন্য","অবধি","অবশ্য","অর্থাত","আই","আগামী","আগে","আগেই","আছে","আজ","আদ্যভাগে","আপনার","আপনি","আবার","আমরা","আমাকে","আমাদের","আমার","আমি","আর","আরও","ই","ইত্যাদি","ইহা","উচিত","উত্তর","উনি","উপর","উপরে","এ","এঁদের","এঁরা","এই","একই","একটি","একবার","একে","এক্","এখন","এখনও","এখানে","এখানেই","এটা","এটাই","এটি","এত","এতটাই","এতে","এদের","এব","এবং","এবার","এমন","এমনকী","এমনি","এর","এরা","এল","এস","এসে","ঐ","ও","ওঁদের","ওঁর","ওঁরা","ওই","ওকে","ওখানে","ওদের","ওর","ওরা","কখনও","কত","কবে","কমনে","কয়েক","কয়েকটি","করছে","করছেন","করতে","করবে","করবেন","করলে","করলেন","করা","করাই","করায়","করার","করি","করিতে","করিয়া","করিয়ে","করে","করেই","করেছিলেন","করেছে","করেছেন","করেন","কাউকে","কাছ","কাছে","কাজ","কাজে","কারও","কারণ","কি","কিংবা","কিছু","কিছুই","কিন্তু","কী","কে","কেউ","কেউই","কেখা","কেন","কোটি","কোন","কোনও","কোনো","ক্ষেত্রে","কয়েক","খুব","গিয়ে","গিয়েছে","গিয়ে","গুলি","গেছে","গেল","গেলে","গোটা","চলে","চান","চায়","চার","চালু","চেয়ে","চেষ্টা","ছাড়া","ছাড়াও","ছিল","ছিলেন","জন","জনকে","জনের","জন্য","জন্যওজে","জানতে","জানা","জানানো","জানায়","জানিয়ে","জানিয়েছে","জে","জ্নজন","টি","ঠিক","তখন","তত","তথা","তবু","তবে","তা","তাঁকে","তাঁদের","তাঁর","তাঁরা","তাঁাহারা","তাই","তাও","তাকে","তাতে","তাদের","তার","তারপর","তারা","তারৈ","তাহলে","তাহা","তাহাতে","তাহার","তিনঐ","তিনি","তিনিও","তুমি","তুলে","তেমন","তো","তোমার","থাকবে","থাকবেন","থাকা","থাকায়","থাকে","থাকেন","থেকে","থেকেই","থেকেও","দিকে","দিতে","দিন","দিয়ে","দিয়েছে","দিয়েছেন","দিলেন","দু","দুই","দুটি","দুটো","দেওয়া","দেওয়ার","দেওয়া","দেখতে","দেখা","দেখে","দেন","দেয়","দ্বারা","ধরা","ধরে","ধামার","নতুন","নয়","না","নাই","নাকি","নাগাদ","নানা","নিজে","নিজেই","নিজেদের","নিজের","নিতে","নিয়ে","নিয়ে","নেই","নেওয়া","নেওয়ার","নেওয়া","নয়","পক্ষে","পর","পরে","পরেই","পরেও","পর্যন্ত","পাওয়া","পাচ","পারি","পারে","পারেন","পি","পেয়ে","পেয়্র্","প্রতি","প্রথম","প্রভৃতি","প্রযন্ত","প্রাথমিক","প্রায়","প্রায়","ফলে","ফিরে","ফের","বক্তব্য","বদলে","বন","বরং","বলতে","বলল","বললেন","বলা","বলে","বলেছেন","বলেন","বসে","বহু","বা","বাদে","বার","বি","বিনা","বিভিন্ন","বিশেষ","বিষয়টি","বেশ","বেশি","ব্যবহার","ব্যাপারে","ভাবে","ভাবেই","মতো","মতোই","মধ্যভাগে","মধ্যে","মধ্যেই","মধ্যেও","মনে","মাত্র","মাধ্যমে","মোট","মোটেই","যখন","যত","যতটা","যথেষ্ট","যদি","যদিও","যা","যাঁর","যাঁরা","যাওয়া","যাওয়ার","যাওয়া","যাকে","যাচ্ছে","যাতে","যাদের","যান","যাবে","যায়","যার","যারা","যিনি","যে","যেখানে","যেতে","যেন","যেমন","র","রকম","রয়েছে","রাখা","রেখে","লক্ষ","শুধু","শুরু","সঙ্গে","সঙ্গেও","সব","সবার","সমস্ত","সম্প্রতি","সহ","সহিত","সাথে","সাধারণ","সামনে","সি","সুতরাং","সে","সেই","সেখান","সেখানে","সেটা","সেটাই","সেটাও","সেটি","স্পষ্ট","স্বয়ং","হইতে","হইবে","হইয়া","হওয়া","হওয়ায়","হওয়ার","হচ্ছে","হত","হতে","হতেই","হন","হবে","হবেন","হয়","হয়তো","হয়নি","হয়ে","হয়েই","হয়েছিল","হয়েছে","হয়েছেন","হল","হলে","হলেই","হলেও","হলো","হাজার","হিসাবে","হৈলে","হোক","হয়"]
display(stop)

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [ ]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

In [ ]:
data['cleantext'] = data.cleantext.apply(lambda x: stopwordRemoval(str(x)))
#display(data.cleantext)
sample_data = range(0,1000)
for i in sample_data:
  print(data.cleantext[i])

ভৌত জগৎ মূলত চারটি উপাদানের সমন্বয়ে তৈরি স্থান কাল সময় ভর শক্তি
বলবিদ্যা বস্তুর বলের ক্রিয়া সংক্রান্ত বিষয় আলোচনা
পদার্থের ধর্ম বস্তুর গুণ আলোচনা
স্থিতিবিদ্যা স্থিতিশীল বস্তুর বলের ক্রিয়া আলোচনা
গতিবিদ্যা গতিশীল বস্তুর বলের ক্রিয়া আলোচনা
ধর্ম সকল পদার্থেরই কমবেশি পদার্থের ধর্ম ওজন বিস্তৃতি রোধ স্থিতিস্থাপকতা
ধর্ম সকল পদার্থের পদার্থের ধর্ম তারতা পাততা দৃঢ়তা ভঙ্গুরতা ধর্ম কেবলমাত্র কঠিন পদার্থের বেলায়
সান্দ্রতা তরল বায়বীয় পদার্থের ধর্ম
পৃষ্ঠটান তলটান তরল পদার্থের ধর্ম
তাপ একপ্রকার শক্তি বস্তুতে প্রয়োগ বস্তুটিকে গরম বস্তুটির তাপমাত্রা বৃদ্ধি পায় বর্জন তাপমাত্রা হ্রাস পায়
তত্ত্ব পরীক্ষানিরীক্ষার সাহায্যে প্রমাণিত মূল কথাগুলি উক্তির প্রকাশ বৈজ্ঞানিক সূত্র
সূত্র সময় আবিষ্কর্তার নামানুসারে ওমের সূত্র বয়েলের সূত্র কখনওবা বিষয়ের নামে শক্তির নিত্যতা সূত্র তাপগতিবিদ্যার সূত্র আবিষ্কারক বিষয় উভয়ের নামে নিউটনের গতিসূত্র গ্যালিলিওর পড়ন্ত বস্তুর সূত্র
সকল প্রাকৃতিক সত্য সরাসরি স্পষ্টভাবে প্রমাণ সত্যের সাহায্যে প্রাকৃতিক ঘটনাকে প্রমাণ নীতি
গাণিতিক মডেল সূত্র প্রতিষ্ঠা লক্ষ্যে পূর্ব

In [ ]:
data["label1"]=data.label1.replace({'Physics':0, 'Chemistry':1, 'Biology':2, 'ICT':3, 'Mathematics':4, 'Others':5})
data["label2"]=data.label2.replace({'Physics':0, 'Chemistry':1, 'Biology':2, 'ICT':3, 'Mathematics':4, 'Others':5})
data["label3"]=data.label3.replace({'Physics':0, 'Chemistry':1, 'Biology':2, 'ICT':3, 'Mathematics':4, 'Others':5})
data["final_label"]=data.final_label.replace({'Physics':0, 'Chemistry':1, 'Biology':2, 'ICT':3, 'Mathematics':4, 'Others':5})
print("First rows of dataset")
data.head()

First rows of dataset


,ID,text,source,cleantext,label1,label2,label3,final_label
0,1,ভৌত জগৎ মূলত চারটি উপাদানের সমন্বয়ে তৈরি- (১)...,HSC Physics,ভৌত জগৎ মূলত চারটি উপাদানের সমন্বয়ে তৈরি স্থা...,0,0,0,0
1,2,বলবিদ্যা বস্তুর উপর বলের ক্রিয়া সংক্রান্ত বিভ...,HSC Physics,বলবিদ্যা বস্তুর বলের ক্রিয়া সংক্রান্ত বিষয় আ...,0,0,0,0
2,3,পদার্থের ধর্ম বস্তুর বিভিন্ন গুণ আলোচনা করে।,HSC Physics,পদার্থের ধর্ম বস্তুর গুণ আলোচনা,0,0,0,0
3,4,স্থিতিবিদ্যা স্থিতিশীল বস্তুর উপর বলের ক্রিয়া...,HSC Physics,স্থিতিবিদ্যা স্থিতিশীল বস্তুর বলের ক্রিয়া আলোচনা,0,0,0,0
4,5,গতিবিদ্যা গতিশীল বস্তুর উপর বলের ক্রিয়া আলোচন...,HSC Physics,গতিবিদ্যা গতিশীল বস্তুর বলের ক্রিয়া আলোচনা,0,0,0,0


In [ ]:
# Drop single or multiple columns by specifying column names
data = data.drop(['ID','text', 'source', 'label1', 'label2', 'label3'], axis=1)
data.head()

,cleantext,final_label
0,ভৌত জগৎ মূলত চারটি উপাদানের সমন্বয়ে তৈরি স্থা...,0
1,বলবিদ্যা বস্তুর বলের ক্রিয়া সংক্রান্ত বিষয় আ...,0
2,পদার্থের ধর্ম বস্তুর গুণ আলোচনা,0
3,স্থিতিবিদ্যা স্থিতিশীল বস্তুর বলের ক্রিয়া আলোচনা,0
4,গতিবিদ্যা গতিশীল বস্তুর বলের ক্রিয়া আলোচনা,0


In [ ]:
# Split the data into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (4200, 2)
Validation data shape: (900, 2)
Testing data shape: (900, 2)


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
from urllib.request import urlopen
import gzip

# get the vectors
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.vec.gz'))

In [ ]:
vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
  values = line.split()
  word = values [0].decode('utf-8')
  vector = np.asarray(values[1:], dtype='float32')
  vocab_and_vectors[word] = vector

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['cleantext'])

In [ ]:
word_index = tokenizer.word_index
vocab_size = len(word_index )+1
print(word_index)
print(vocab_size)

{'মান': 1, 'এক': 2, 'নির্দিষ্ট': 3, 'সংখ্যা': 4, 'বস্তুর': 5, 'তৈরি': 6, 'বল': 7, 'সমান': 8, 'তড়িৎ': 9, 'ধরনের': 10, 'ডেটা': 11, 'সময়': 12, 'সৃষ্টি': 13, 'উৎপন্ন': 14, 'সকল': 15, 'রাসায়নিক': 16, 'শক্তি': 17, 'পরিমাণ': 18, 'দুইটি': 19, 'অর্থাৎ': 20, 'ব্যবহৃত': 21, 'একক': 22, 'ইলেকট্রন': 23, 'পরিবর্তন': 24, 'গঠিত': 25, 'বৃদ্ধি': 26, 'ঘটে': 27, 'হিসেবে': 28, 'বস্তু': 29, 'যুক্ত': 30, 'কম': 31, 'ভিন্ন': 32, 'বিন্দু': 33, 'পদার্থের': 34, 'বিন্দুতে': 35, 'মূল': 36, 'বিন্দুর': 37, 'প্রতিটি': 38, 'চৌম্বক': 39, 'প্রকাশ': 40, 'গঠন': 41, 'পদার্থ': 42, 'কোষ': 43, 'বিপরীত': 44, 'ক্রিয়া': 45, 'অবস্থান': 46, 'শূন্য': 47, 'স্থির': 48, 'তথ্য': 49, 'অবস্থিত': 50, 'কম্পিউটার': 51, 'দশমিক': 52, 'বড়': 53, 'ভেক্টর': 54, 'একাধিক': 55, 'গ্যাস': 56, 'বেগ': 57, 'বলের': 58, 'নির্ণয়': 59, 'এসিড': 60, 'জৈব': 61, 'পরমাণুর': 62, 'ধনাত্মক': 63, 'থাকলে': 64, 'পায়': 65, 'অংশ': 66, 'দিক': 67, 'নেটওয়ার্ক': 68, 'ফাংশন': 69, 'উপাদান': 70, 'তরঙ্গ': 71, 'যেকোনো': 72, 'পরিণত': 73, 'তাপমাত্রায়': 74, 'যৌগ': 75, 'যেসব':

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in word_index.items():
  embedding_vector = vocab_and_vectors.get(word)
  # words that cannot be found will be set to 0
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
max_len = 40
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['cleantext'])
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# validation Sequences
val_sequences = tokenizer.texts_to_sequences(val_data['cleantext'])
val_pad_sequences =  keras.preprocessing.sequence.pad_sequences(val_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,val_pad_sequences.shape)

# test Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['cleantext'])
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

12954
Number of Training Sequences : (4200, 40)
Number of Validation Sequences : (900, 40)
Number of Test Sequences : (900, 40)


In [ ]:
num_classes = 6
train_labels = tf.keras.utils.to_categorical(train_data['final_label'], num_classes)
val_labels = tf.keras.utils.to_categorical(val_data['final_label'], num_classes)
test_labels = tf.keras.utils.to_categorical(test_data['final_label'], num_classes)

In [ ]:
# Enable CUDA support in TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        # Enable memory growth to allocate GPU memory on an as-needed basis
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [ ]:
# Define the CNN model architecture
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False, input_length = max_len))
cnn_model.add(Conv1D(128, 5, activation='relu'))
cnn_model.add(MaxPooling1D())
cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
cnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

cnn_model.summary()

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

# Train the model
history = cnn_model.fit(train_pad_sequences, train_labels, validation_data=(val_pad_sequences, val_labels), epochs=30, batch_size=32,
    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_model.evaluate(test_pad_sequences, test_labels)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

# Get the predicted labels
predictions = cnn_model.predict(test_pad_sequences)
predicted_labels = predictions.argmax(axis=1)

true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics
test_precision = precision_score(true_labels, predicted_labels, average='weighted')
test_recall = recall_score(true_labels, predicted_labels, average='weighted')
test_f1 = f1_score(true_labels, predicted_labels, average='weighted')
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print the results
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Confusion Matrix:")
print(confusion_mat)

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_52 (Embedding)    (None, 40, 300)           3886500   
                                                                 
 conv1d_29 (Conv1D)          (None, 36, 128)           192128    
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 18, 128)          0         
 1D)                                                             
                                                                 
 dropout_41 (Dropout)        (None, 18, 128)           0         
                                                                 
 flatten_7 (Flatten)         (None, 2304)              0         
                                                                 
 dense_65 (Dense)            (None, 128)               295040    
                                                     

In [ ]:
# Define the BiLSTM model architecture
bilstm_model = Sequential()
bilstm_model.add(Embedding(vocab_size, 300,weights=[embedding_matrix],trainable=False, input_length = max_len))
bilstm_model.add(Bidirectional(LSTM(128, return_sequences=True)))
bilstm_model.add(MaxPooling1D())
bilstm_model.add(Dropout(0.2))
bilstm_model.add(Flatten())
bilstm_model.add(Dense(128, activation='relu'))
bilstm_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
bilstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

bilstm_model.summary()

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

# Train the model
history = bilstm_model.fit(train_pad_sequences, train_labels, validation_data=(val_pad_sequences, val_labels), epochs=30, batch_size=32,
    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = bilstm_model.evaluate(test_pad_sequences, test_labels)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

# Get the predicted labels
predictions = cnn_model.predict(test_pad_sequences)
predicted_labels = predictions.argmax(axis=1)

true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics
test_precision = precision_score(true_labels, predicted_labels, average='weighted')
test_recall = recall_score(true_labels, predicted_labels, average='weighted')
test_f1 = f1_score(true_labels, predicted_labels, average='weighted')
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print the results
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Confusion Matrix:")
print(confusion_mat)

Model: "sequential_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_50 (Embedding)    (None, 40, 300)           3886500   
                                                                 
 bidirectional_30 (Bidirecti  (None, 40, 256)          439296    
 onal)                                                           
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 20, 256)          0         
 1D)                                                             
                                                                 
 dropout_39 (Dropout)        (None, 20, 256)           0         
                                                                 
 flatten_6 (Flatten)         (None, 5120)              0         
                                                                 
 dense_63 (Dense)            (None, 128)             

In [ ]:
# Define the CNN+BiLSTM model architecture
cnn_bilstm_model = Sequential()
cnn_bilstm_model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False, input_length = max_len))
cnn_bilstm_model.add(Conv1D(128, 5, activation='relu'))
cnn_bilstm_model.add(MaxPooling1D())
cnn_bilstm_model.add(Bidirectional(LSTM(128, return_sequences=True)))
cnn_bilstm_model.add(Dropout(0.2))
cnn_bilstm_model.add(Flatten())
cnn_bilstm_model.add(Dense(128, activation='relu'))
cnn_bilstm_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

cnn_bilstm_model.summary()

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

# Train the model
history = cnn_bilstm_model.fit(train_pad_sequences, train_labels, validation_data=(val_pad_sequences, val_labels), epochs=30, batch_size=32,
    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_bilstm_model.evaluate(test_pad_sequences, test_labels)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

# Get the predicted labels
predictions = cnn_model.predict(test_pad_sequences)
predicted_labels = predictions.argmax(axis=1)

true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics
test_precision = precision_score(true_labels, predicted_labels, average='weighted')
test_recall = recall_score(true_labels, predicted_labels, average='weighted')
test_f1 = f1_score(true_labels, predicted_labels, average='weighted')
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print the results
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Confusion Matrix:")
print(confusion_mat)

Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_61 (Embedding)    (None, 40, 300)           3886500   
                                                                 
 conv1d_40 (Conv1D)          (None, 36, 128)           192128    
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 18, 128)          0         
 g1D)                                                            
                                                                 
 bidirectional_39 (Bidirecti  (None, 18, 256)          263168    
 onal)                                                           
                                                                 
 dropout_50 (Dropout)        (None, 18, 256)           0         
                                                                 
 flatten_16 (Flatten)        (None, 4608)            